## This Notebook used to Map the schema comming from the control_df or res dataframe
>> Also It'll check the files or folder exists or not in the Raw Layer

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from datetime import datetime 


In [0]:
## run the setup notebook 




In [0]:
%run /Workspace/Generic/Nb_SetUpMounts_JDBC

[FileInfo(path='dbfs:/FileStore/2024_04_18_4_WheelSales-1.csv', name='2024_04_18_4_WheelSales-1.csv', size=10992761, modificationTime=1719993217000),
 FileInfo(path='dbfs:/FileStore/2024_04_18_4_WheelSales.csv', name='2024_04_18_4_WheelSales.csv', size=10992762, modificationTime=1719989742000),
 FileInfo(path='dbfs:/FileStore/4_WheelSales.csv', name='4_WheelSales.csv', size=10992762, modificationTime=1719989774000),
 FileInfo(path='dbfs:/FileStore/jars/', name='jars/', size=0, modificationTime=1716203553000),
 FileInfo(path='dbfs:/FileStore/source1.csv', name='source1.csv', size=1138, modificationTime=1709555634000),
 FileInfo(path='dbfs:/FileStore/tables/', name='tables/', size=0, modificationTime=1708600418000)]

['PipelineName', 'Pk_PipelineID', 'Pk_ActivityID', 'PipelineSourceTypes', 'ParameterValueSink', 'parameterValues2', 'ColumnSourceTypes', 'SourceColumnName', 'TargetColumnName', 'sourcecolumntypes', 'IsNullable', 'IsActive']


PipelineName,Pk_PipelineID,Pk_ActivityID,PipelineSourceTypes,ParameterValueSink,parameterValues2,ColumnSourceTypes,SourceColumnName,TargetColumnName,sourcecolumntypes,IsNullable,IsActive
PL_Copy2WheelsData,101,2,CSV,2-WheelSales.csv,2-WheelSData.parquet,CSV,"[OEM, Model, Segment, Month, Year, NoOfSales, EngineCapacity, Mileage, Model Price, Dealer location, Region]","[OEM, Model, Segment, Month, Year, NoOfSales, EngineCapacity, Mileage, ModelPrice, Dealerlocation, Region]","[string, string, string, string, int, int, string, string, string, string, string]",0,1
PL_Copy4WheelsData,102,3,CSV,4-WheelSales.csv,4-WheelSData.parquet,CSV,"[Brand, Model, Body Type, FUEL TYPE, Mileage, ENGINE, TRANSMISSION, Price range, Fuel Capacity (L), Seating Capacity, Top_Speed in Km/h, No Of Cylinders, Year Of Sale, Month Of Sale , Country, City]","[Brand, Model, BodyType, FUELTYPE, Mileage, ENGINE, TRANSMISSION, Pricerange, FuelCapacity, SeatingCapacity, Top_Speed, NoOfCylinders, YearOfSale, MonthOfSale, Country, City]","[string, string, string, string, string, string, string, string, int, int, string, int, int, string, string, string]",null,null


PipelineName,Pk_PipelineID,Pk_ActivityID,PipelineSourceTypes,ParameterValueSink,parameterValues2,ColumnSourceTypes,SourceColumnName,TargetColumnName,sourcecolumntypes,IsNullable,IsActive,test
PL_Copy2WheelsData,101,2,CSV,2-WheelSales.csv,2-WheelSData.parquet,CSV,"[OEM, Model, Segment, Month, Year, NoOfSales, EngineCapacity, Mileage, Model Price, Dealer location, Region]","[OEM, Model, Segment, Month, Year, NoOfSales, EngineCapacity, Mileage, ModelPrice, Dealerlocation, Region]","[string, string, string, string, int, int, string, string, string, string, string]",0,1,2-WheelSales
PL_Copy4WheelsData,102,3,CSV,4-WheelSales.csv,4-WheelSData.parquet,CSV,"[Brand, Model, Body Type, FUEL TYPE, Mileage, ENGINE, TRANSMISSION, Price range, Fuel Capacity (L), Seating Capacity, Top_Speed in Km/h, No Of Cylinders, Year Of Sale, Month Of Sale , Country, City]","[Brand, Model, BodyType, FUELTYPE, Mileage, ENGINE, TRANSMISSION, Pricerange, FuelCapacity, SeatingCapacity, Top_Speed, NoOfCylinders, YearOfSale, MonthOfSale, Country, City]","[string, string, string, string, string, string, string, string, int, int, string, int, int, string, string, string]",null,null,4-WheelSales


2-WheelSales
/mnt/genericSilver/2-WheelSales
abfss://xyenta-dev-96-silver@adlsxyentadevuks96.dfs.core.windows.net/2-WheelSales/
4-WheelSales
/mnt/genericSilver/4-WheelSales
abfss://xyenta-dev-96-silver@adlsxyentadevuks96.dfs.core.windows.net/4-WheelSales/


2-WheelSales
/mnt/genericrawTest/2-WheelSales
abfss://xyenta-dev-96-raw@adlsxyentadevuks96.dfs.core.windows.net/2-WheelSales/
4-WheelSales
/mnt/genericrawTest/4-WheelSales
abfss://xyenta-dev-96-raw@adlsxyentadevuks96.dfs.core.windows.net/4-WheelSales/


2-WheelSales
/mnt/genericStage/2-WheelSales
abfss://xyenta-dev-96-stg@adlsxyentadevuks96.dfs.core.windows.net/2-WheelSales/
4-WheelSales
/mnt/genericStage/4-WheelSales
abfss://xyenta-dev-96-stg@adlsxyentadevuks96.dfs.core.windows.net/4-WheelSales/


In [0]:
# Define the function for schema mapping

2024/07/04


In [0]:
def schema_mappings(filepaths, reference_df):
    try:
        # Define a mapping for types
        type_dict = {
            'string': StringType(),
            'int': IntegerType(),
            'long': LongType(),
            'float': FloatType(),
            'double': DoubleType(),
            'boolean': BooleanType(),
            'date': DateType(),
            'timestamp': TimestampType()
        }

        for row in reference_df.collect():
            filenames = row['parameterValues2']
            print(f"\nProcessing files: {filenames}")
            folder_names = row['test']
            print(folder_names)

            source_columns = row['SourceColumnName']
            target_columns = row['TargetColumnName']
            target_columns = target_columns.strip("[]").split(",")
            print(f'Target_before:: {target_columns}')
            target_columns = [col_name.strip() for col_name in target_columns]

            source_types = row['sourcecolumntypes']
            source_types = source_types.strip("[]").split(",")
            source_types = [col_type.strip() for col_type in source_types]

            if len(target_columns) != len(source_types):

                raise ValueError(f"Mismatch between target columns and source types lengths for {filenames}")

            type_mapping = {}

            for col_name, col_type in zip(target_columns, source_types):

                if col_type not in type_dict:
                    raise ValueError(f"Unsupported data type: {col_type}")
                type_mapping[col_name] = type_dict[col_type]


            print(f'{type_mapping}')

            dt_string = datetime.now().strftime("%Y/%m/%d")
            full_path = f"{filepaths}{folder_names}/{dt_string}/{filenames}"
            print(full_path)

            try:
                dbutils.fs.ls(full_path)
                print(f'path exists for :: {full_path}')

                dfs = spark.read.format('parquet').load(full_path)

                actual_columns = dfs.columns

                print(f'Actual_columns for {filenames} are :: {actual_columns}')

                for col_name, col_type in type_mapping.items():
                    if col_name in actual_columns:
                        dfs = dfs.withColumn(col_name, col(col_name).cast(col_type))

                    else:
                        raise ValueError(f"Column '{col_name}' from target columns does not exist in the DataFrame")

                dfs.printSchema()

                output_good_path = f"/mnt/genericSilver/{folder_names}/{dt_string}"
                dfs.coalesce(1).write.mode('overwrite').option("header", "true").parquet(output_good_path)
                print("Done")

            except Exception as e :
                print(f'Does not exists :: {full_path}')

    except Exception as e:
        raise e

In [0]:
schema_mappings(filepaths='/mnt/genericraw/', reference_df= res)


Processing files: 2-WheelSData.parquet
2-WheelSales
Target_before:: ['OEM', ' Model', ' Segment', ' Month', ' Year', ' NoOfSales', ' EngineCapacity', ' Mileage', ' ModelPrice', ' Dealerlocation', ' Region']
{'OEM': StringType(), 'Model': StringType(), 'Segment': StringType(), 'Month': StringType(), 'Year': IntegerType(), 'NoOfSales': IntegerType(), 'EngineCapacity': StringType(), 'Mileage': StringType(), 'ModelPrice': StringType(), 'Dealerlocation': StringType(), 'Region': StringType()}
/mnt/genericraw/2-WheelSales/2024/07/04/2-WheelSData.parquet
path exists for :: /mnt/genericraw/2-WheelSales/2024/07/04/2-WheelSData.parquet
Actual_columns for 2-WheelSData.parquet are :: ['Dealerlocation', 'EngineCapacity', 'Mileage', 'Model', 'ModelPrice', 'Month', 'NoOfSales', 'OEM', 'Region', 'Segment', 'Year']
root
 |-- Dealerlocation: string (nullable = true)
 |-- EngineCapacity: string (nullable = true)
 |-- Mileage: string (nullable = true)
 |-- Model: string (nullable = true)
 |-- ModelPrice: 